<a href="https://colab.research.google.com/github/bhuvana-ak/huggingface_experiments/blob/main/MNA_LLM_NB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install trafilatura transformers peft bitsandbytes trl deepeval

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.0/367.0 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 8.7 MB/s eta 0:00:00
   

In [ ]:
import os

In [ ]:
from google.colab import drive
# drive.flush_and_unmount()
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
os.chdir("/content/drive/MyDrive/MNA_Model_NoteBook")

In [ ]:
!pwd

/content/drive/MyDrive/MNA_Model_NoteBook


In [ ]:
!ls

mna_dataset_csv.csv


Read all csv and create one master csv after cleaning up data such as date formats,

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd

mna_deals = pd.read_csv('mna_dataset_csv.csv')

In [ ]:
mna_deals.head()

,article_text,input_url,announced_date_og,announced_date_new,announced_date,acquiring_company,acquired_company,deal_amount,mna_status,mna_type,output
0,NaN,https://www.prnewswire.com:443/news-releases/b...,2/25/2022,02/25/2022,02/25/2022,Biohaven Pharmaceuticals,Knopp Biosciences,NaN,Announced,Acquisition,NaN
1,NaN,https://www.globenewswire.com/news-release/202...,2/25/2022,02/25/2022,02/25/2022,Castellum,Lexington Solutions Group,NaN,Announced,Acquisition,NaN
2,NaN,http://www.businesswire.com/news/home/20220225...,2/25/2022,02/25/2022,02/25/2022,Chatham Asset Management,RR Donnelley,NaN,Completed,Acquisition,NaN
3,NaN,https://www.prnewswire.com:443/news-releases/3...,2/25/2022,02/25/2022,02/25/2022,3G Capital,Hunter Douglas,NaN,Completed,Acquisition,NaN
4,NaN,https://www.prnewswire.com:443/news-releases/r...,2/25/2022,02/25/2022,02/25/2022,Riviana Foods,InHarvest,NaN,Announced,Acquisition,NaN


In [ ]:
def create_output(row):
  data = {
      "announced_date" : row['announced_date'],
      "acquiring_company" : row['acquiring_company'],
      "acquired_company" : row['acquired_company'],
      "deal_amount" : 0 if str(row['deal_amount']) == 'nan' else row['deal_amount'],
      "mna_type" : row['mna_type']
  }
  return data

# Apply the function to each row
mna_deals['output'] = mna_deals.apply(create_output, axis=1)


In [ ]:
mna_deals['output'][:1][0]

{'announced_date': '02/25/2022',
 'acquiring_company': 'Biohaven Pharmaceuticals',
 'acquired_company': 'Knopp Biosciences',
 'deal_amount': 0,
 'mna_type': 'Acquisition'}

In [ ]:
def generate_article_text(row):
  if str(row['article_text']=='nan'):
    # scrape data
    return ""
  else:
    return row['article_text']

# Apply the function to each row
mna_deals['article_text'] = mna_deals.apply(generate_article_text, axis=1)


In [ ]:
mna_deals['article_text'][:1][0]

''

In [ ]:
url = "https://www.prnewswire.com/news-releases/biohaven-acquires-kv7-ion-channel-platform-for-the-treatment-of-epilepsy-and-other-neurologic-disorders-from-channel-biosciences-a-subsidiary-of-knopp-biosciences-301490378.html"
downloaded = fetch_url(url)
result = extract(downloaded, output_format="markdown", with_metadata=True, no_fallback=True)

#### New code Aug 27
Read csv, create a json format with keys - instruction, input, output

In [1]:
import os
import pandas as pd

from google.colab import drive
# drive.flush_and_unmount()
drive.mount('/content/drive')

os.chdir("/content/drive/MyDrive/MNA_Model_NoteBook")

train_data = pd.read_csv('mna_train_data.csv')

display(train_data)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Unnamed: 0,input,output
0,0,# Biohaven Acquires Kv7 Ion Channel Platform f...,"{'announced_date': '02/25/2022', 'acquiring_co..."
1,1,"POTOMAC, Md., Feb. 25, 2022 (GLOBE NEWSWIRE) -...","{'announced_date': '02/25/2022', 'acquiring_co..."
2,2,CHICAGO--(BUSINESS WIRE)--Chatham Asset Manage...,"{'announced_date': '02/25/2022', 'acquiring_co..."
3,3,"ROTTERDAM, Netherlands and NEW YORK, Feb. 25, ...","{'announced_date': '02/25/2022', 'acquiring_co..."
4,4,# RIVIANA FOODS TO ACQUIRE ASSETS OF DUININCK ...,"{'announced_date': '02/25/2022', 'acquiring_co..."
...,...,...,...
954,965,Cambrex Announces Sale of Drug Product Busines...,"{'announced_date': nan, 'acquiring_company': n..."
955,966,Automakers oppose Cleveland-Cliffs push to buy...,"{'announced_date': nan, 'acquiring_company': n..."
956,967,Vietnam's FPT acquires US product engineering ...,"{'announced_date': nan, 'acquiring_company': n..."
957,968,Ex­clu­sive: BioN­Tech ac­quires lipid nanopar...,"{'announced_date': nan, 'acquiring_company': n..."


In [2]:
data = []
for _, row in train_data.iterrows():
  instruction = "Extract the following from the input M&A news article: announced_date (The date when the M&A deal was announced), acquiring_company(The name of the acquiring company), acquired_company(The name of the acquired company), deal_amount(The deal amount of the merger or acquisition), mna_type(The type of M&A deal: Acquisition or Merger)"
  input = row['input']
  output = row['output']
  sample = {"instruction": instruction, "input": input, "output": output}
  data.append(sample)



In [3]:
from datasets import Dataset
new_dataset = Dataset.from_list(data)

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
new_dataset.push_to_hub("bhuvana-ak7/mna_training_v3", private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/bhuvana-ak7/mna_training_v3/commit/3cd464d2b7445f908c793b2df11ada08bcc1e786', commit_message='Upload dataset', commit_description='', oid='3cd464d2b7445f908c793b2df11ada08bcc1e786', pr_url=None, pr_revision=None, pr_num=None)